In [ ]:
%load_ext autoreload
%autoreload 2
from RTKToArrayConversion import *
from ExtendedConeBeamDCC import *
from TextFileSaving2 import * 
from AllAcquisitionCDClass import *
from tqdm import tqdm_notebook

In [ ]:
filesdir_ref = "/home/mmouchet/Documents/SIEMENSDATA/GO.SIM/phantom/14_01_22/4D/sans_table/donneesBrutes/P007_Trot035_Tresp0/"
# reading projections
proj = itk.imread(filesdir_ref+"sub36_proj_865_980.mha")
# Reading the geometry of the scanner
xmlreader = rtk.ThreeDCircularProjectionGeometryXMLFileReader.New()
xmlreader.SetFilename(filesdir_ref+'sub36_geometry_865_980.xml')
# xmlreader.SetFilename(filesdir_ref+"geometry.xml")
xmlreader.GenerateOutputInformation()
geometry = xmlreader.GetOutputObject()
print('nproj = %d'%(len(geometry.GetGantryAngles())))
print(proj.GetLargestPossibleRegion().GetSize())

In [ ]:
#Convert to array for faster computation
geometry_array = RTKtoNP(geometry)
proj_array = itk.GetArrayFromImage(proj)
proj_infos = GetProjectionInformations(proj)
source_pos_array = GetSourcePositions(geometry)
rotation_matrices_array = GetRotationMatrices(geometry)
fixed_matrices_array = GetFixedSystemMatrices(geometry)
print(proj_infos)
n_proj_per_rotation = np.where(geometry_array[2, :] == geometry_array[2, 0])[0][1] - np.where(geometry_array[2, :] == geometry_array[2, 0])[0][0]
print(n_proj_per_rotation)

In [ ]:
# first_idx = np.where(geometry_array[8,:]<-296)[0][0]
# last_idx = np.where(geometry_array[8,:]>-395)[0][-1]
# print(first_idx, last_idx)
# T_rot = 0.35 #s
# T_step = 0.1 #s
# idx_proj_ref = np.arange(first_idx, last_idx)
# idx_proj_ref = np.arange(proj_infos[2][2])
idx_proj_ref = np.array(np.where(geometry_array[2, :] <= np.abs(geometry_array[2, 0]- geometry_array[2, 1]))[0])
print(len(idx_proj_ref))
# axial_limit =  int(100*np.round((1-1/32)/0.07))//2
# print(axial_limit)

In [ ]:
#compute dcc
AcquiDCC = DCCOnCDinAnAcquisition(geometry_array, source_pos_array, rotation_matrices_array, fixed_matrices_array, proj_array, proj_infos)
# import multiprocessing as mp
# with mp.Pool(4) as pool:
#     jobs = [pool.apply(ComputeDCCFromOneRef, args=(argument,)) for argument in zip(idx_proj_ref, [geometry_array]*len(idx_proj_ref), [source_pos_array]*len(idx_proj_ref), [rotation_matrices_array]*len(idx_proj_ref), [fixed_matrices_array]*len(idx_proj_ref), [proj_array]*len(idx_proj_ref), [proj_infos]*len(idx_proj_ref))]
#     pool.close()
#     res = []
#     for job in tqdm_notebook(jobs):
#         res.append(job.get())
# from joblib import Parallel, delayed
# import time
# start = time.time()
# res = Parallel(n_jobs=11, backend = "multiprocessing")(delayed(ComputeDCCFromOneRef)(ref, geometry_array, source_pos_array, rotation_matrices_array, fixed_matrices_array, proj_array, proj_infos) for ref in tqdm_notebook(idx_proj_ref))
# print(time.time()-start)

In [ ]:
# AcquiDCC.ComputeAllPossiblePairsForEachPos(idx_proj_ref, 'False')

In [ ]:
AcquiDCC.ComputeDCCForEachPosPara(idx_proj_ref, 'False')

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
%matplotlib notebook
def motion(t):
    return 0.1608 + 0.936*np.cos(2*np.pi*t-np.pi) + 0.2808*np.cos(4*np.pi*t-3*np.pi/4) + 0.03744*np.cos(6*np.pi*t - np.pi/3) + 0.03744*np.cos(8*np.pi*t+np.pi/2) +0.013104*np.cos(10*np.pi*t+np.pi)

# Fréquence d'échantillonnage
fe = 72/0.5  # Hz
# Fréquence de nyquist
f_nyq = fe / 2.  # Hz
# Fréquence de coupure
fc = 0.025*fe  # Hz
# Préparation du filtre de Butterworth en passe-bas
b, a = signal.butter(4, fc/f_nyq, 'low', analog=False)


# plt.figure()
# plt.plot(AcquiDCC.res[44][1], AcquiDCC.res[44][2])
# plt.show()

# yf = fft([np.median(AcquiDCC.res[i]) for i in range(len(idx_proj_ref))])
# xf = fftfreq(len(idx_proj_ref), 0.5/36)[:len(idx_proj_ref)//2]
# plt.figure()
# plt.plot(xf, 2.0/len(idx_proj_ref) * np.abs(yf[0:len(idx_proj_ref)//2]))
# plt.grid()
# # plt.xlabel("Frequency (Hz)")
# # plt.ylabel("F(Med(E))")
# plt.show()

In [ ]:
savedir = "/home/mmouchet/Documents/SIEMENSDATA/GO.SIM/phantom/14_01_22/4D/sans_table/donneesBrutes/P007_Trot035_Tresp4/"
WriteAllPairsErrorsFile(savedir + "Tresp4_sub36_angle0", AcquiDCC.res)

In [ ]:
ref, idx, mom0, mom1 = ReadAllPairsErrorsFile(savedir + "Tresp4_sub36_angle0")

In [ ]:

savedir = "/home/mmouchet/Documents/SIEMENSDATA/GO.SIM/phantom/14_12_21/3D/stats_report/"

refs_ib, res_ib = ReadAllPairsErrorsFile(savedir + "ib_sub72_n380_n250_ad_HannWover10")
refs_rl, res_rl = ReadAllPairsErrorsFile(savedir + "ib_corr_sub72_n380_n250_ad_HannWover10")

%matplotlib notebook

plt.figure()
plt.plot(geometry_array[8,refs_ib], [np.median(res_ib[i]) for i in range(len(refs_ib))], linewidth=0.5, label = "original sinogram")
# # Fréquence d'échantillonnage
# fe = 360/0.5  # Hz
# # Fréquence de nyquist
# f_nyq = fe / 2.  # Hz
# # Fréquence de coupure
# fc = 0.025*fe  # Hz
# # Préparation du filtre de Butterworth en passe-bas
# b, a = signal.butter(4, fc/f_nyq, 'low', analog=False)
# plt.plot(refs_ib*0.5/360,signal.filtfilt(b, a, [np.median(res_ib[i]) for i in range(len(refs_ib))]),'k')
plt.plot(geometry_array[8,refs_rl], [np.median(res_rl[i]) for i in range(len(refs_rl))], linewidth=0.5, label = "corrected sinogram")
# # Fréquence d'échantillonnage
# fe = 360/0.5  # Hz
# # Fréquence de nyquist
# f_nyq = fe / 2.  # Hz
# # Fréquence de coupure
# fc = 0.025*fe  # Hz
# # Préparation du filtre de Butterworth en passe-bas
# b, a = signal.butter(4, fc/f_nyq, 'low', analog=False)
# plt.plot(refs_rl*0.5/360,signal.filtfilt(b, a, [np.median(res_rl[i]) for i in range(len(refs_rl))]),'g')
plt.xlabel("z(mm)")
plt.ylabel("Med(E)")
plt.legend()
plt.show()

# proj_ar = itk.GetArrayFromImage(proj)

# plt.figure()
# plt.imshow(proj_ar[109:721,16,:].T, cmap='gray', origin = "lower",extent=(geometry_array[8,refs_rl[-1]],geometry_array[8,refs_rl[0]],AcquiDCC.gamma[0]*180/np.pi,AcquiDCC.gamma[-1]*180/np.pi))
# plt.ylabel("Fan angle (°)")
# plt.xlabel("z(mm)")
# ax = plt.twinx()
# ax.plot(geometry_array[8,refs_rl],[np.median(res_ib[i]) for i in range(len(refs_ib))],'r',linewidth = 0.5, label = "Med(E)")
# plt.legend()
# plt.ylabel("Med(E)")
# plt.show()

In [ ]:
# Read all pair moments
refs, res_4D = ReadAllPairsErrorsFile("/home/mmouchet/PycharmProjects/4D_study/bpm_12/stats_bpm12/apm_bpm12_Hann_Wover5_AbsDiff.csv")
plt.figure()
plt.plot(refs*0.35/36, [np.median(res_4D[i]) for i in range(len(refs))])
s_but = signal.filtfilt(b, a, [np.median(res_4D[i]) for i in range(len(refs))])
plt.plot(refs*0.35/36, s_but)
plt.ylabel("Med(E)")
plt.xlabel("Time (s)")
ax = plt.twinx()
ax.plot(refs*0.35/36, np.abs(motion(refs/514)), color = 'red', label = 'breathing motion')
# ax.plot(refs*0.35/36, np.abs(np.gradient(motion(refs/514))),'r', label = 'breathing motion gradient')
ax.legend(loc="upper right")
plt.yticks([])
# ax.plot(np.arange(len(idx_proj_ref))*0.5/36, geometry_array[2,idx_proj_ref]*180/np.pi, 'r--')
plt.show()

yf = fft([np.median(res_4D[i]) for i in range(len(refs))])
xf = fftfreq(len(refs), 0.35/36)[:len(refs)//2]
plt.figure()
plt.plot(xf, 2.0/len(refs) * np.abs(yf[0:len(refs)//2]))
plt.grid()
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("F(Med(E))")
plt.show()

In [ ]:
# Read all pair moments
refs, res_4D = ReadAllPairsErrorsFile("/home/mmouchet/PycharmProjects/4D_study/bpm_12/stats_bpm12/apm_bpm12_Hann_Wover5_Diff.csv")
plt.figure()
# off =  int(1.2*36/0.35)
# [len(res_4D[i])//2-off:len(res_4D[i])//2+off]
plt.plot(refs*0.35/36, [np.median(res_4D[i]) for i in range(len(refs))])
s_but = signal.filtfilt(b, a, [np.median(res_4D[i]) for i in range(len(refs))])
plt.plot(refs*0.35/36, s_but)
plt.ylabel("Med(E)")
plt.xlabel("Time (s)")
ax = plt.twinx()
ax.plot(refs*0.35/36, -motion(refs/514), color = 'red', label = 'breathing motion')
# ax.plot(refs*0.35/36, np.gradient(motion(refs/514)),'r', label = 'breathing motion gradient')
ax.legend(loc="upper right")
plt.yticks([])
# ax.plot(np.arange(len(idx_proj_ref))*0.5/36, geometry_array[2,idx_proj_ref]*180/np.pi, 'r--')
plt.show()

yf = fft([np.median(res_4D[i]) for i in range(len(refs))])
xf = fftfreq(len(refs), 0.35/36)[:len(refs)//2]
plt.figure()
plt.plot(xf, 2.0/len(refs) * np.abs(yf[0:len(refs)//2]))
plt.grid()
# plt.xlabel("Frequency (Hz)")
# plt.ylabel("F(Med(E))")
plt.show()